# Gaussian Boson Sampling (GBS) benchmark

Expanded notebook: exact probabilities via SF Fock backend, sampling with Gaussian and Fock backends, KL divergence and plots.

In [ ]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import json
import strawberryfields as sf
from strawberryfields.ops import Sgate, Interferometer, MeasureFock
from thewalrus import hafnian
from scripts.quantum_photonics.run_gbs import build_interferometer, compute_exact_probs_fock, sample_gbs, kl_between_empirical_and_exact
print('numpy', np.__version__)
print('strawberryfields', sf.__version__)

In [ ]:
# Configuration
modes = 4
squeezing = 0.6
shots = 200
cutoff = 6
rng_seed = 42
np.random.seed(rng_seed)
U = build_interferometer(modes, seed=rng_seed)
print('U shape', U.shape)

In [ ]:
# Exact probabilities (Fock backend)
exact = compute_exact_probs_fock(modes=modes, squeezing=squeezing, U=U, cutoff=cutoff)
nonzero = sorted((k,v) for k,v in exact.items() if v>0)[:10]
print('example nonzero probs', nonzero[:5])

In [ ]:
# Sampling and KL calculation
samples_gauss = sample_gbs(modes=modes, squeezing=squeezing, U=U, backend='gaussian', shots=shots)
samples_fock = sample_gbs(modes=modes, squeezing=squeezing, U=U, backend='fock', shots=shots, cutoff=cutoff)
kl_gauss = kl_between_empirical_and_exact(samples_gauss, exact, cutoff=cutoff)
kl_fock = kl_between_empirical_and_exact(samples_fock, exact, cutoff=cutoff)
print('KL(gaussian || exact)=', kl_gauss)
print('KL(fock || exact)=', kl_fock)

In [ ]:
# Visualize total photon counts
def totals(samples):
    return [int(sum(s)) for s in samples]
plt.hist(totals(samples_gauss), bins=range(0, 1 + max(totals(samples_gauss) + totals(samples_fock))), alpha=0.6, label='gaussian')
plt.hist(totals(samples_fock), bins=range(0, 1 + max(totals(samples_gauss) + totals(samples_fock))), alpha=0.6, label='fock')
plt.legend(); plt.title('Total photon counts (gaussian vs fock)'); plt.show()

In [ ]:
# Save results summary
out = {'modes':modes, 'squeezing':squeezing, 'shots':shots, 'kl_gauss':float(kl_gauss), 'kl_fock':float(kl_fock)}
open('bundles/v23_toe_finish/v23/gbs_benchmark_results.json','w').write(json.dumps(out, indent=2))
print('Saved gbs_benchmark_results.json')

# GBS threshold sweep results (extended)

This section loads the extended threshold-sweep results and displays the JS vs eta plot and a short summary.


In [ ]:
import json
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import Image, display

repo = Path.cwd()
png = repo / 'bundles' / 'v23_toe_finish' / 'v23' / 'gbs_threshold_js_vs_eta_extended.png'
json_f = repo / 'bundles' / 'v23_toe_finish' / 'v23' / 'gbs_threshold_sweep_extended.json'

if png.exists():
    display(Image(str(png)))
else:
    print('Plot not found yet:', png)

if json_f.exists():
    data = json.load(open(json_f))
    # show top 6 rows by JS
    top = sorted(data, key=lambda r: r['js'], reverse=True)[:6]
    import pandas as pd
    df = pd.DataFrame(top)
    display(df)
else:
    print('JSON results not found yet:', json_f)
